In [ ]:
from IPython.core.display import HTML
with open ("../style.css", "r") as file:
    css = file.read()
HTML(css)

# A Shift-Reduce Parser for Arithmetic Expressions

In this notebook we implement a generic *shift reduce parser*.  The parse table that we use 
implements the following grammar for arithmetic expressions:
$$
  \begin{eqnarray*}
  \mathrm{expr}        & \rightarrow & \mathrm{expr}\;\;\texttt{'+'}\;\;\mathrm{product}   \\
                       & \mid        & \mathrm{expr}\;\;\texttt{'-'}\;\;\mathrm{product}   \\
                       & \mid        & \mathrm{product}                                    \\[0.2cm]
  \mathrm{product}     & \rightarrow & \mathrm{product}\;\;\texttt{'*'}\;\;\mathrm{factor} \\
                       & \mid        & \mathrm{product}\;\;\texttt{'/'}\;\;\mathrm{factor} \\
                       & \mid        & \mathrm{factor}                                     \\[0.2cm]
  \mathrm{factor}      & \rightarrow & \texttt{'('} \;\;\mathrm{expr} \;\;\texttt{')'}     \\
                       & \mid        & \texttt{NUMBER} 
  \end{eqnarray*}
$$

In [ ]:
%load_ext nb_mypy

## Implementing a Scanner

In order to parse, we need a scanner.  We will use a scanner that is similar to the one that we have already used for our *top down parser* discussed earlier in Chapter 7.

In [ ]:
import re

The function `tokenize` scans the string `s` into a list of tokens using Python's regular expressions.  The scanner distinguishes between
* whitespace, which is discarded,
* numbers,
* arithmetical operators and parenthesis,
* all remaining characters, which are treated as lexical errors.

See below for an example.

In [ ]:
def tokenize(s: str) -> list[str]:
    '''Transform the string s into a list of tokens.  The string s
       is supposed to represent an arithmetic expression.
    '''
    lexSpec = r'''([ \t\n]+)      |  # blanks and tabs
                  ([1-9][0-9]*|0) |  # number
                  ([-+*/()])      |  # arithmetical operators and parentheses 
                  (.)                # unrecognized character
               '''
    tokenList = re.findall(lexSpec, s, re.VERBOSE)
    result    = []
    for ws, number, operator, error in tokenList:
        if ws:        # skip blanks and tabs
            continue
        elif number:
            result += [ 'NUMBER' ]
        elif operator:
            result += [ operator ]
        else:
            result += [ f'ERROR({error})']
    return result

In [ ]:
tokenize('1 + 2 * (3 - 4)')

In [ ]:
Rule       = tuple[str, tuple[str, ...]]
Action     = str | tuple[str, str] | tuple[str, Rule]
actionTable: dict[tuple[str, str], Action]
gotoTable:   dict[tuple[str, str], str]

Assume a grammar $G = \langle V, T, R, S \rangle$ is given.  A  *shift-reduce parser*
is defined as a 4-Tuple
$$P = \langle Q, q_0, \texttt{action}, \texttt{goto} \rangle$$
where
- $Q$ is the set of *states* of the shift-reduce parser.  

  For the purpose of the shift-reduce-parser, states are purely abstract. 
- $q_0 \in Q$ is the *start state*.
- $\texttt{action}$ is a function taking two arguments. The first argument is a state $q \in Q$
  and the second argument is a token $t \in T$.  The result of this function is an element from the set
  $$\texttt{Action} :=
       \bigl\{ \langle\texttt{shift}, q\rangle  \mid q \in Q \bigr\}               \cup 
       \bigl\{ \langle\texttt{reduce}, r\rangle \mid r \in R \bigr\} \cup 
       \bigl\{ \texttt{accept} \bigr\}                        \cup
       \bigl\{ \texttt{error}  \bigr\}.                         
  $$
  Here `shift`, `reduce`, `accept`, and `error` are strings that serve to
  distinguish the different kinds of results returned by the function 
  `action`.  Therefore the signature of the function `action` is given as follows:
  $$\texttt{action}: Q \times T \rightarrow \texttt{Action}.$$
- `goto` is a function that takes a state $q \in Q$ and a syntactical variable
  $v \in V$ and computes a new state.  Therefore the signature of `goto` is as follows:
  $$\texttt{goto}: Q \times V \rightarrow Q.$$

The class `ShiftReduceParser` maintains two tables that are implemented as dictionaries:
- `mActionTable` encodes the function $\texttt{action}: Q \times T \rightarrow \texttt{Action}$.
- `mGotoTable` encodes the function $\texttt{goto}: Q \times V \rightarrow Q$.

The constructor takes these tables as arguments and stores them in the member variables `mActionTable` and `mGotoTable`.

In [ ]:
class ShiftReduceParser():
    def __init__(self, actionTable:dict[tuple[str, str], Action], 
                       gotoTable: dict[tuple[str, str], str]) -> None:
        self.mActionTable: dict[tuple[str, str], Action] = actionTable
        self.mGotoTable:   dict[tuple[str, str], str]    = gotoTable
        
    def parse(self, TL: list[str]) -> bool:
        return None # type: ignore

The method `parse` takes a list of tokens `TL` as its argument.  It returns `True` if the token list can be parsed successfully or `False` otherwise.  
The algorithm that is applied is known as *shift/reduce parsing*.

In [ ]:
def parse(self, TL: list[str]) -> bool:
    index              = 0      # points to next token
    Symbols: list[str] = []     # stack of symbols, i.e. tokens or variables
    States:  list[str] = ['s0'] # stack of states, s0 is start state
    TL += ['EOF']
    while True:
        q = States[-1]
        t = TL[index]
        # Below, an undefined table entry is interpreted as an error entry.
        match self.mActionTable.get((q, t), 'error'):
            case 'error': 
                return False
            case 'accept':
                return True
            case 'shift', s: 
                Symbols += [t]
                States  += [s]
                index   += 1
            case 'reduce', rule:
                head, body = rule
                n       = len(body)
                Symbols = Symbols[:-n]
                States  = States [:-n] 
                Symbols = Symbols + [head]
                state   = States[-1]
                States += [ self.mGotoTable[state, head] ]
            
ShiftReduceParser.parse = parse # type: ignore
del parse

## Testing

In [ ]:
%run Parse-Table.ipynb

In [ ]:
def test(s:str) -> None: 
    parser = ShiftReduceParser(actionTable, gotoTable)
    TL     = tokenize(s)
    if parser.parse(TL):
        print('Parse successful!')
    else:
        print('Parse failed!')

In [ ]:
test('(1 + 2) * 3')

In [ ]:
test('1 * 2 + 3 * (4 - 5) / 2')

In [ ]:
test('11+22*(33-44)/(5-10*5/(4-3))')

In [ ]:
test('1+2*3-')